### import

In [1]:
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
from datetime import datetime, timedelta
import os, re, pickle
from dateutil.relativedelta import relativedelta
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle, Font, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
import warnings
warnings.simplefilter("ignore")

##################################
company = "솔림"      # 솔림 or 대성
basedate = "240731" 
새채무자키사용 = True
##################################
dir = join(r"D:\3.자산\전산 dataset", company, basedate)

원데이터파일명 = {"개회차주" : "2."+ company +"_개인회생_원데이터_차주"+basedate, 
                "개회보증인" : "2."+ company +"_개인회생_원데이터_보증인"+basedate, 
                "신복차주" : "2."+ company +"_신용회복_원데이터_차주"+basedate, 
                "신복보증인" : "2."+ company +"_신용회복_원데이터_보증인"+basedate, 
                "전체" : "2."+ company +"_전체_원데이터_"+basedate}

if 새채무자키사용 : 
    raw_data = pd.read_pickle(join(dir, "pkl", 원데이터파일명["전체"]+"_by새채무자키.pkl"))
else : 
    raw_data = pd.read_pickle(join(dir, "pkl", 원데이터파일명["전체"]+".pkl"))

def save_df_to_excel_underline(df, fullpath, key_columns_no=1, font_size=9):
    """키값의 열번호를 작성하면 키값이 달라질때마다 밑줄 그어줌(a열이 1번), 필요없으면 0입력"""
    # DataFrame을 엑셀 파일로 저장
    df.to_excel(fullpath, index=False, sheet_name='Sheet1')

    # 엑셀 파일 열기
    workbook = load_workbook(fullpath)
    sheet = workbook['Sheet1']
    
    # 날짜 포맷 스타일 정의
    date_style = NamedStyle(name='date_style', number_format='yyyy-mm-dd')
    
    # 날짜 타입인 열에 대해 형식 적용
    for column in df.select_dtypes(include=['datetime64']).columns:
        col_idx = df.columns.get_loc(column) + 1  # Excel은 1-based index
        for row in range(2, sheet.max_row + 1):  # 데이터가 2번째 행부터 시작
            cell = sheet.cell(row=row, column=col_idx)
            cell.style = date_style

    # 글씨 크기 변경 (폰트 설정)
    font = Font(size=font_size)
    
    # 모든 셀에 대해 글씨 크기 설정
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        for cell in row:
            cell.font = font
        
    # '주민등록번호' 열의 값이 달라질 때마다 밑줄 추가
    if key_columns_no > 0 : 
        underline_border = Border(bottom=Side(style='thin'))
        prev_value = None
                
        for row in range(2, sheet.max_row + 1):
            current_value = sheet.cell(row=row, column=key_columns_no).value  
            if prev_value is not None and current_value != prev_value:
                for col in range(1, sheet.max_column + 1):
                    sheet.cell(row=row-1, column=col).border = underline_border
            prev_value = current_value        
    

    # 엑셀 파일 저장
    workbook.save(fullpath)
    
def 파일명(폴더:str, 키워드:str) :
    file_list = os.listdir(폴더)
    account_filename = [file for file in file_list if re.search(키워드, file)]
    if len(account_filename) == 1 :
        return join(폴더,account_filename[0])
    else :
        print("해당 키워드가 들어간 파일이 하나가 아닙니다.")
    

### 종결건 제거

In [2]:
미종결건 = raw_data.query('~새채무상태.str.match("종료")')

### 상태 및 담당자 다건

In [44]:
다건cols = ["주민등록번호","새채무자키","채무자키","계좌키", "새채무상태","채무상태","담당자","매각사구분","원채권사","채권구분","메모"]

In [26]:
주민번호그룹 = 미종결건.sort_values(by=['주민등록번호','채무자키','계좌키']).reset_index(drop=True).groupby(by=['주민등록번호'])
다건cols = ["주민등록번호","새채무자키","채무자키","계좌키", "새채무상태","채무상태","담당자","채권구분","메모"]

새채무상태다건 = 주민번호그룹.filter(lambda x: x.새채무상태.nunique()>1)[다건cols]
채무상태다건 = 주민번호그룹.filter(lambda x: x.채무상태.nunique() > 1)[다건cols]
담당자다건 = 주민번호그룹.filter(lambda x: x.담당자.nunique() > 1)[다건cols]

if not os.path.exists(join(dir,"check")) : 
    os.mkdir(join(dir,"check"))

save_df_to_excel_underline(새채무상태다건, join(dir, "check", basedate+"_[check]새채무상태다건.xlsx"))
save_df_to_excel_underline(채무상태다건, join(dir, "check", basedate+"_[check]채무상태다건.xlsx"))
save_df_to_excel_underline(담당자다건, join(dir, "check", basedate+"_[check]담당자다건.xlsx"))

### 새채무상태 vs 채무상태 : 조정절차관련

In [49]:
# 조정채권 아닌 경우는 어차피 채무상태와 담당자를 바탕으로 새채무상태를 정하므로 실사가 필요.
# 파산, 사망 데이터 확보하면 코드 보완
x1 = 미종결건.새채무상태
x2 = 미종결건.채무상태
미종결건["상태합"] = 미종결건.새채무상태 + 미종결건.채무상태

# 일치조건
개회확정조건 = (x1=="개인회생(확정)")&(x2=="개인회생(확정)")
개회진행조건 = (x1=="개인회생(진행중)")&(x2=="개인회생(진행중)")
신용회복진행중조건 = (x1=="신용회복(진행중)")&((x2=="신용회복(진행중)")|(x2=="신용회복(확정)")|(x2=="신용회복(개인)")|(x2=="신용회복(프리)")) # 기존 확정자의 재조정절차는 굳이 수정하지 않는다.
신용회복개인조건 = (x1=="신용회복(개인)")&((x2=="신용회복(개인)")|(x2=="신용회복(확정)"))
신용회복프리조건 = (x1=="신용회복(프리)")&((x2=="신용회복(프리)")|(x2=="신용회복(확정)"))

조정채권상태값일치조건 = (개회확정조건|개회진행조건|신용회복진행중조건|신용회복개인조건|신용회복프리조건)

조정채권이면수정 = 미종결건[~조정채권상태값일치조건]

# 일치조건 뺐으면 이제 조정상태값은 없어야 함. 그런데 새채무상태나 채무상태에 조정값이 발견되는 경우는 수정대상
조정상태값불일치 = 조정채권이면수정[조정채권이면수정.상태합.str.contains("개인회생|신용회복")][다건cols+["보증인상태"]]

# 중복 제거 : 상태값까지 완전히 중복일때
조정상태값불일치 = 조정상태값불일치.drop_duplicates(subset=['채무자키','새채무상태','채무상태'])

save_df_to_excel_underline(조정상태값불일치, join(dir, "check", basedate+"_[check]조정상태값불일치.xlsx"), 0)

### 정상채무자 있는데 미배정건
- 채무상태, 보증인상태 정비 후, 차주와 보증인을 대등한 채무자로 놓고

In [3]:
# 담당자 배정은 채무자키 단위로 진행
# 1) 정상보증인 있으면 가능(보증인 불가건은 제외함)
if 새채무자키사용 :
    x = 미종결건["새보증인상태by새채무자키"]
else : 
    x = 미종결건["새보증인상태"]

# 2) 차주정상(추심가능)
차주신용회복cond = (미종결건.새채무상태.str.contains("신용회복") & ~미종결건.새채무상태.str.contains("누락"))
차주정상cond = ((미종결건.새채무상태=="개인")|(미종결건.새채무상태=="시효")) # 보증인정상, 상각, 종결처리요망은 pass
보증인정상cond = x.str.contains("정상|누락|폐지|실효")
conds = [차주신용회복cond, 차주정상cond, 보증인정상cond]
values = ["불가", "가능", "가능"]
default_value = "불가"
미종결건["추심가능여부"] = np.select(conds, values, default_value)

정상채무자cols = ["주민등록번호","새채무자키","채무자키","계좌키", "새채무상태","채무상태","담당자","보증인상태","새보증인상태","매각사구분","원채권사","메모"] # 새채무자키는 일부러 뺌
정상채무자 = 미종결건.query('추심가능여부=="가능"')[정상채무자cols]
정상채무자 = 정상채무자.sort_values(["주민등록번호","새채무자키","채무자키"])


# 담당자파일 읽기                                   #############################
path_담당자 = join(r"D:\3.자산\기타 업무\담당자변경", "사원_20240814_1308.xlsx")
                                                   #############################
담당자 = pd.read_excel(path_담당자, dtype={"사원키":str})[["사원키", "부서","사원성명"]]
담당자 = 담당자.query('부서=="추심" and ~사원성명.str.contains("테스트")') # 배정(대기), 배정(유의)는 제외

# 채무자읽기 (담당자키 사용위해)
path_debt = 파일명(join(dir,"pkl"), "채무자조회")
debt = pd.read_pickle(path_debt)[["채무자키","담당자키"]]

# 담당자키 merge
정상채무자 = 정상채무자.merge(debt, on=["채무자키"], how='left')

# 담당자 미배정건만 남기기
정상채무자 = 정상채무자.merge(담당자[["사원키","부서"]], left_on="담당자키", right_on="사원키", how='left')
정상채무자 = 정상채무자.query('부서!="추심" and ~담당자.str.contains("정호석|정명선|정현화|정필공|배정\(")') # 정호석, 배정대기건 제외
정상채무자 = 정상채무자[정상채무자cols]

# 출력 : 출력 후 담당자가 추심팀인건 제외하고 보면 됨
save_df_to_excel_underline(정상채무자, join(dir, "check", basedate+"_[check]정상채무자있음_배정확인.xlsx"), key_columns_no=1)